## Imports

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="5"

import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import pickle
import numpy as np


batch_size = 512
num_classes = 10
img_rows, img_cols = 28, 28

Using TensorFlow backend.


## Load data

In [2]:
X_train, y_train, X_train_skel_features = None, None, None
with open("data/train_info", "rb") as fin:
    data = pickle.load(fin)
    X_train, y_train, X_train_skel_features = data["data"], data["labels"], data["skel_features"]

X_test, y_test, X_test_skel_features = None, None, None
with open("data/test_info", "rb") as fin:
    data = pickle.load(fin)
    X_test, y_test, X_test_skel_features = data["data"], data["labels"], data["skel_features"]    

## Create data for network

In [3]:
x_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
x_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## Create f1 score

In [4]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

## Generate model

In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy', f1])

## Create callbacks

In [6]:
early_stopping = EarlyStopping(monitor='val_acc', mode='max', patience=5, verbose=1)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', mode='max', factor=0.5, patience=3, verbose=1)
callbacks=[early_stopping, model_checkpoint, reduce_lr]

## Fit model

In [7]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10000,
          callbacks=callbacks,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10000
60000/60000 [==============================] - 11s 180us/step - loss: 1.6019 - acc: 0.4261 - f1: 0.3232 - val_loss: 0.5978 - val_acc: 0.8435 - val_f1: 0.7671

Epoch 00001: val_acc improved from -inf to 0.84350, saving model to best_model.h5
Epoch 2/10000
60000/60000 [==============================] - 3s 44us/step - loss: 0.7594 - acc: 0.7422 - f1: 0.7384 - val_loss: 0.2755 - val_acc: 0.9221 - val_f1: 0.9200

Epoch 00002: val_acc improved from 0.84350 to 0.92210, saving model to best_model.h5
Epoch 3/10000
60000/60000 [==============================] - 3s 45us/step - loss: 0.5561 - acc: 0.8179 - f1: 0.8175 - val_loss: 0.1733 - val_acc: 0.9481 - val_f1: 0.9506

Epoch 00003: val_acc improved from 0.92210 to 0.94810, saving model to best_model.h5
Epoch 4/10000
60000/60000 [==============================] - 3s 48us/step - loss: 0.4561 - acc: 0.8550 - f1: 0.8554 - val_loss: 0.1557 - val_acc: 0.9524 - val_f1: 0.9530

Epoch 00004:


Epoch 00033: val_acc did not improve from 0.98220

Epoch 00033: ReduceLROnPlateau reducing learning rate to 0.0625.
Epoch 34/10000
60000/60000 [==============================] - 3s 47us/step - loss: 0.1907 - acc: 0.9431 - f1: 0.9435 - val_loss: 0.0566 - val_acc: 0.9816 - val_f1: 0.9821

Epoch 00034: val_acc did not improve from 0.98220
Epoch 35/10000
60000/60000 [==============================] - 3s 46us/step - loss: 0.1885 - acc: 0.9422 - f1: 0.9431 - val_loss: 0.0566 - val_acc: 0.9820 - val_f1: 0.9822

Epoch 00035: val_acc did not improve from 0.98220
Epoch 36/10000
60000/60000 [==============================] - 3s 46us/step - loss: 0.1894 - acc: 0.9430 - f1: 0.9442 - val_loss: 0.0568 - val_acc: 0.9821 - val_f1: 0.9825

Epoch 00036: val_acc did not improve from 0.98220

Epoch 00036: ReduceLROnPlateau reducing learning rate to 0.03125.
Epoch 00036: early stopping


## Score model

In [8]:
model = load_model('best_model.h5', custom_objects={'f1': f1})
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test f1_score:', score[2])

Test loss: 0.05756415659706108
Test accuracy: 0.9822
Test f1_score: 0.9816322724342346
